# Files Aggregator

This script is used for market data combination and serialisation into a pickle file, for ease and faster use of the data for training the model. Use this script **after** the data has been aggregated and cleaned through a bash script, named `CSVCombiner.sh`.

In [25]:
import pandas as pd
import pickle

Function, global variable, and source & target absolute path declaration

**Change** the absolute path when necessary

In [26]:
# data absolute path
source_path='../new_data/.staging/'
source_prefix = 'HK'
source_postfix = '_cleaned.csv'
target_path='../new_data/Processed/'
target_postfix = '.pkl'

# column number in the data (csv file) which is used: date, high, low, close and open price, and trade volume
cols = list(range(6))

# helper function
def parser(x):
    try:
        try:
            return pd.datetime.strptime(x, '%Y-%m-%d %H:%M')
        except: 
            return pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
    except:
        return pd.datetime.strptime(x, '%Y/%m/%d %H:%M')

Stocks going to be used declaration. Both company code (HSBC, HSE, TCEHY for Tencent) and its ticker are needed as the data is named after the ticker. Make sure that data has been aggregated using the bash script (present in the `[project_root_directory]/new_data/.staging/` directory ). 

Each item in the *tickers* should correspond to each item in the *companyCode* in the same order (i.e. i<sup>th</sup> *tickers* item is the ticker for i<sup>th</sup> *companyCode* item)

**Change** the desired stocks (ticker and company code) as necessary

In [27]:
# i.e. 'TCEHY': d_0700, 'CNOOC': d_0883, 'HSBC': d_0005, 'HSE': d_0011, 'AIA': d_1299, 'BOC': d_3988
tickers = ['0011', '3988', '1299']
companyCode=['HSE', 'BOC', 'AIA']

In [28]:
# retrieving and combining the data into a single list, and renamed the column header accordingly
#     index_col specifies that the date and time to be the index
#     parse_dates and date_parser normalise the format of the date and time to be consistent
market_data = []
for ticker in tickers:
    print("Getting stock HK", ticker, "...")
    stock_data = pd.read_csv(source_path+source_prefix+ticker+source_postfix, usecols=cols, index_col=[0], 
                             parse_dates=[0], date_parser=parser)
    stock_data = stock_data.rename(index=str, columns={"Open": "adj_open", "Close": "adj_close","High": "adj_high",
                                                       "Low": "adj_low", "Volume": "adj_volume"})
    cols_ = stock_data.columns.tolist()
    market_data.append(stock_data[[cols_[0], cols_[3], cols_[2], cols_[1], cols_[4]]])
print('Data retrieval done! \n')

# sanity check
df=market_data[0]
print(df.head(3))
print(df.tail(3))

Getting stock HK 0011 ...
Getting stock HK 3988 ...
Getting stock HK 1299 ...
Data retrieval done! 

                     adj_open  adj_close  adj_high  adj_low  adj_volume
Date                                                                   
2015-01-02 09:21:00     129.1      129.1     129.1    129.1        6000
2015-01-02 09:31:00     129.1      129.1     129.2    129.0        3610
2015-01-02 09:32:00     129.2      129.2     129.2    129.2         100
                     adj_open  adj_close  adj_high  adj_low  adj_volume
Date                                                                   
2017-12-29 15:58:00     194.1      194.2     194.1    194.2          41
2017-12-29 15:59:00     194.1      194.2     194.1    194.2          50
2017-12-29 16:00:00     194.2      194.0     194.0    194.2         723


In [29]:
# change the data placeholder into a pandas Panel as it provides easy access for 3D data.
#    First dimension / item: company stock (i.e. HSE, AIA)
#    Second dimension / major_axis: data point at a single discrete date and time (i.e. prices at a particular time)
#    Third dimension / minor_axis: data type (i.e. closing price, volume traded, high price)
data = {code: data_ for code, data_ in zip(companyCode, market_data)}
data = pd.core.panel.Panel(data)

# sanity check
print(data)
print(data.items)

<class 'pandas.core.panel.Panel'>
Dimensions: 3 (items) x 234429 (major_axis) x 5 (minor_axis)
Items axis: HSE to AIA
Major_axis axis: 2015-01-02 09:21:00 to 2017-12-29 16:00:00
Minor_axis axis: adj_open to adj_volume
Index(['HSE', 'BOC', 'AIA'], dtype='object')


Define the serialised file name. The default is `market_data_[all_company_codes`.

**Change** the file name as necessary, make sure to use `.pkl` extension at the end of the file name.

In [39]:
data_name = "market_data"
for stock in companyCode:
    data_name += '_' + stock

# specify the file type
data_name += target_postfix

print("file directory:", target_path+data_name)

file directory: ../new_data/Processed/market_data_HSE_BOC_AIA.pkl


In [36]:
# save the data in the form of pandas' Panel as pickle file
data.to_pickle(target_path+data_name)

# sanity check
with open(target_path+data_name, "rb") as input:
    mdata = pickle.load(input)
print(mdata)
print(mdata.items)

<class 'pandas.core.panel.Panel'>
Dimensions: 3 (items) x 234429 (major_axis) x 5 (minor_axis)
Items axis: HSE to AIA
Major_axis axis: 2015-01-02 09:21:00 to 2017-12-29 16:00:00
Minor_axis axis: adj_open to adj_volume
Index(['HSE', 'BOC', 'AIA'], dtype='object')
